# Extract text

**Import PDFMiner**

In [376]:
from pdfminer.layout import LAParams
import pandas as pd

In [377]:
from pdfminer.high_level import extract_text
def extract_text_from_pdf(pdf_path):
    laparams = LAParams()
    text = extract_text(pdf_path,laparams)
    return text.split("\f")

In [378]:
extract_text_from_pdf('Q3-2024-Press-Release-English.pdf')

['Canadian Tire Corporation Reports Third Quarter 2024 Results; Announces Annual \n\nDividend Increase for 15th Consecutive Year and Share Repurchase Intention \n\nToronto, November 7, 2024 – Canadian Tire Corporation, Limited (TSX:CTC, TSX: CTC.A) (CTC \nor the Company) today released its third quarter results for the period ended September 28, 2024.  \n\n•  Consolidated  comparable  sales1  trend  improved  compared  to  Q2  2024;  consolidated \n\ncomparable sales were down 1.5% compared to Q3 2023.  \n\n•  Diluted and Normalized Earnings Per Share1 (EPS) were $3.59, compared to $(1.19) in Q3 \n\n2023, and up 21.3% from $2.96 on a normalized basis. \n\n•  Annualized  dividend  increased  from  $7.00  to  $7.10  per  share,  alongside  an  intention  to \n\nrepurchase up to $200.0 million of Class A Non-Voting Shares in 2025.  \n\n“We delivered strong retail profitability for the third consecutive quarter and sales trends improved,” \n\nsaid  Greg  Hicks,  President  and  CEO,  Canad

## Extract text across pages

In [379]:
import re
def normalize_text(text):
    return re.sub(r'\s+', ' ', text.strip()).lower()

def extract_text_between_words_across_pages(text_pages, start_word, end_word):
    # Normalize words
    start_word = normalize_text(start_word)
    end_word = normalize_text(end_word)
    extracted_texts = []
    combined_text = "\n".join([normalize_text(page) for page in text_pages])
    pattern = re.compile(re.escape(start_word) + '(.*?)' + re.escape(end_word), re.S)

    for match in pattern.finditer(combined_text):
        start_page, end_page = None, None
        start_index, end_index = match.start(), match.end()

        cumulative_length = 0
        for i, page in enumerate(text_pages):
            page_length = len(page)
            if start_page is None and start_index < cumulative_length + page_length:
                start_page = i + 1
            if end_page is None and end_index <= cumulative_length + page_length:
                end_page = i + 1
            cumulative_length += page_length + len("\n")
            if start_page is not None and end_page is not None:
                break

        extracted_texts.append({
            'start_page': start_page,
            'end_page': end_page,
            'start_word': start_word,
            'end_word': end_word,
            'extracted_text': match.group(1).strip()
        })
    return extracted_texts


Extract text by sections

In [380]:
start_end_pairs = [
    ('THIRD-QUARTER HIGHLIGHTS', 'products are set to roll out during 2025.'),
    ('CONSOLIDATED OVERVIEW','Company in the quarter.'),
    ('RETAIL SEGMENT OVERVIEW','impacted the Retail segment in the quarter.'),
    ('FINANCIAL SERVICES OVERVIEW','Services segment in the quarter.'),
    ('CT REIT OVERVIEW','November 5, 2024.'),
    ('CAPITAL ALLOCATION','million to $575 million.'),
    ('QUARTERLY DIVIDEND','for tax purposes.'),
    ('SHARE REPURCHASES','to regulatory approvals.'),
    ('1) NON-GAAP FINANCIAL MEASURES','Owned brand penetration'),
    ('FORWARD-LOOKING STATEMENTS', 'by applicable securities laws.'),
    ('CONFERENCE CALL','at this website for 12 months.'),
    ('ABOUT CANADIAN TIRE CORPORATION','visit Corp.CanadianTire.ca.'),
    ('FOR MORE INFORMATION','karen.keyes@cantire.com')
    # Add more pairs as needed
]

import os
def process_files_in_folder(file_name, start_end_pairs):
    all_extracted_texts = []
    if file_name.endswith('.pdf'):
      text_pages = extract_text_from_pdf(file_name)
      for start_word, end_word in start_end_pairs:
        extracted_texts = extract_text_between_words_across_pages(text_pages, start_word, end_word)
        all_extracted_texts.extend(extracted_texts)

    return all_extracted_texts

all_extracted_texts = process_files_in_folder('Q3-2024-Press-Release-English.pdf', start_end_pairs)

In [381]:
all_extracted_texts

[{'start_page': 1,
  'end_page': 3,
  'start_word': 'third-quarter highlights',
  'end_word': 'products are set to roll out during 2025.',
  'extracted_text': '• consolidated comparable sales were down 1.5%; sportchek grew for the first quarter since q2 2023, which partially offset declines at canadian tire retail (ctr) and mark’s. o ctr comparable sales1 were down 2.2%, compared to q3 2023. customers continued to prioritize essential categories including automotive, which continued to perform well against a strong quarter in q3 2023, led by growth in automotive service. o sportchek comparable sales1 were up 2.9%, marking two consecutive quarters in which sportchek outperformed industry trends. targeted promotional events and\nimproved customer experience continued to be a focus and contributed to growth in athletic footwear and hockey categories. o mark’s comparable sales1 were down 2.3%, led by industrial wear declines, which were partially offset by growth in men’s shorts and t-shir

# Text summarizer
Summarize text by section
Lets Create a text summarizer using Cosine Distance

In [382]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import re

def read_article(article):
    sentences = []
    for sentence in article.split('. '):
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split())
    return sentences

def sentence_similarity(sent1, sent2, stop_words=None):
    if stop_words is None:
        stop_words = []

    sent1 = [w.lower() for w in sent1 if w not in stop_words]
    sent2 = [w.lower() for w in sent2 if w not in stop_words]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # Build the vector for the first sentence
    for w in sent1:
        if w in stop_words:
            continue
        vector1[all_words.index(w)] += 1

    # Build the vector for the second sentence
    for w in sent2:
        if w in stop_words:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:  # ignore if both are the same sentence
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

def generate_summary(article, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text and split it
    sentences = read_article(article)

    # Step 2 - Generate Similarity Matrix across sentences
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity matrix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    print("Indexes of top ranked_sentence order are ", ranked_sentence)

    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Output the summarize text
    print("Summarized Text: \n", ". ".join(summarize_text))
    return summarize_text


In [383]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Andrea
[nltk_data]     FS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Testing summarizer in sections

In [384]:
for section in all_extracted_texts:
    print(f"Summary for section starting with '{section['start_word']}' and ending with '{section['end_word']}':")
    generate_summary(section['extracted_text'], top_n=2)
    print("\n")

Summary for section starting with 'third-quarter highlights' and ending with 'products are set to roll out during 2025.':
Indexes of top ranked_sentence order are  [(0.07653128487177972, ['•', 'consolidated', 'comparable', 'sales', 'were', 'down', '1.5%;', 'sportchek', 'grew', 'for', 'the', 'first', 'quarter', 'since', 'q2', '2023,', 'which', 'partially', 'offset', 'declines', 'at', 'canadian', 'tire', 'retail', '(ctr)', 'and', 'mark’s']), (0.07537800341474762, ['•', 'improved', 'retail', 'profitability', 'led', 'to', 'higher', 'consolidated', 'income', 'before', 'income', 'taxes', '(ibt)', 'at', '$299.3', 'million,', 'an', 'increase', 'of', '$230.0', 'million', 'and', '$33.0', 'million', 'on', 'a', 'normalized', 'basis1', 'compared', 'to', 'the', 'prior', 'year']), (0.06414638177243122, ['•', 'in-store', 'net', 'promoter', 'score', '(nps)', 'was', 'up', 'across', 'the', 'company’s', 'banners,', 'including', 'ctr;', 'store', 'investments', 'and', 'a', 'focus', 'on', 'strong', 'in-stock

## Saving and structuring summarizations

In [385]:
import json

processed_data = {
    'document_sections': []
}

for section in all_extracted_texts:
    processed_data['document_sections'].append({
        'start_word': section['start_word'],
        'end_word': section['end_word'],
        'summary': generate_summary(section['extracted_text'], top_n=1),
        'full_text': section['extracted_text']
    })



Indexes of top ranked_sentence order are  [(0.07653128487177972, ['•', 'consolidated', 'comparable', 'sales', 'were', 'down', '1.5%;', 'sportchek', 'grew', 'for', 'the', 'first', 'quarter', 'since', 'q2', '2023,', 'which', 'partially', 'offset', 'declines', 'at', 'canadian', 'tire', 'retail', '(ctr)', 'and', 'mark’s']), (0.07537800341474762, ['•', 'improved', 'retail', 'profitability', 'led', 'to', 'higher', 'consolidated', 'income', 'before', 'income', 'taxes', '(ibt)', 'at', '$299.3', 'million,', 'an', 'increase', 'of', '$230.0', 'million', 'and', '$33.0', 'million', 'on', 'a', 'normalized', 'basis1', 'compared', 'to', 'the', 'prior', 'year']), (0.06414638177243122, ['•', 'in-store', 'net', 'promoter', 'score', '(nps)', 'was', 'up', 'across', 'the', 'company’s', 'banners,', 'including', 'ctr;', 'store', 'investments', 'and', 'a', 'focus', 'on', 'strong', 'in-stock', 'availability', 'of', 'key', 'brands', 'continued', 'to', 'drive', 'improvements', 'in', 'positive', 'customer', 'senti

In [386]:
processed_data

{'document_sections': [{'start_word': 'third-quarter highlights',
   'end_word': 'products are set to roll out during 2025.',
   'summary': ['• consolidated comparable sales were down 1.5%; sportchek grew for the first quarter since q2 2023, which partially offset declines at canadian tire retail (ctr) and mark’s'],
   'full_text': '• consolidated comparable sales were down 1.5%; sportchek grew for the first quarter since q2 2023, which partially offset declines at canadian tire retail (ctr) and mark’s. o ctr comparable sales1 were down 2.2%, compared to q3 2023. customers continued to prioritize essential categories including automotive, which continued to perform well against a strong quarter in q3 2023, led by growth in automotive service. o sportchek comparable sales1 were up 2.9%, marking two consecutive quarters in which sportchek outperformed industry trends. targeted promotional events and\nimproved customer experience continued to be a focus and contributed to growth in athlet

In [387]:
# Save to a JSON file for chatbot usage
with open('processed_document.json', 'w') as file:
    json.dump(processed_data, file, indent=4)

# Scrapping tables

Tables on pages: 6,7,8,9 and 10 (3 tables on page 7)

In [388]:
import locale

In [389]:
# Function to set locale safely
def set_locale(locale_name):
    try:
        locale.setlocale(locale.LC_ALL, locale_name)
    except locale.Error:
        print(f"Locale {locale_name} not supported on this system. Using default locale.")
        locale.setlocale(locale.LC_ALL, '')

# Set the locale (adjust as necessary for your system)
set_locale('en_US.UTF-8')  # Common locale setting for Unix-like systems
locale._override_localeconv = {'n_sign_posn':1}

In [390]:
# Format as currency
def format_currency(value):
    if pd.isna(value):
        return ''
    return locale.currency(value, grouping=True).split('.')[0]

import tabula


## Extract tables

In [391]:
#another way : Tabula - First Step
import tabula

# Path to the PDF file
pdf_path = 'Q3-2024-Press-Release-English.pdf'
table = tabula.read_pdf(pdf_path, pages=[6,7,8,9,10], multiple_tables=True)

## Separate and organize them

### Table 1 - Page 6

In [392]:
table1 = table[0]
table1

,Net income,$,220.7 $,(27.8) $,540.2 $,141.9
0,Net income attributable to shareholders,NaN,200.6,(66.4),476.2,40.8
1,Add normalizing items:,NaN,NaN,NaN,NaN,NaN
2,DC fire expense (recovery),$,— $,(96.4) $,— $,8.4
3,GST/HST-related charge1,NaN,—,—,—,24.7
4,Change in fair value of redeemable financial i...,NaN,—,328.0,—,328.0
5,Normalized Net income,$,220.7 $,203.8 $,540.2 $,503.0
6,Normalized Net income attributable to sharehol...,$,200.6 $,165.2 $,476.2 $,396.9
7,Normalized Diluted EPS,$,3.59 $,2.96 $,8.54 $,7.0


we drop the extra column

In [393]:
table1.drop(columns='$', inplace=True)

All of the tables are omiting the column names and instead are using the first row as such.
For this we create a function to add the names of the columns and use the first column as the index

In [394]:
def structure_table(table, new_columns):
    # Step 1: Move the current header to the first row
    table = table.rename(columns={i: col for i, col in enumerate(table.iloc[0])})
    table.loc[-1] = list(table.columns)  # Insert the header row as the first row
    table.index = table.index + 1       # Adjust the index
    table = table.sort_index()          # Re-sort by index

    # Step 2: Rename the columns
    table.columns = new_columns
    
    return table


In [395]:
new_columns = ['(C$ in millions)','Q3_2024', 'Q3_2023', 'YTD_Q3_2024', 'YTD_Q3_2023']
table1 = structure_table(table1, new_columns)
print("\nModified DataFrame:")
table1


Modified DataFrame:


,(C$ in millions),Q3_2024,Q3_2023,YTD_Q3_2024,YTD_Q3_2023
0,Net income,220.7 $,(27.8) $,540.2 $,141.9
1,Net income attributable to shareholders,200.6,(66.4),476.2,40.8
2,Add normalizing items:,NaN,NaN,NaN,NaN
3,DC fire expense (recovery),— $,(96.4) $,— $,8.4
4,GST/HST-related charge1,—,—,—,24.7
5,Change in fair value of redeemable financial i...,—,328.0,—,328.0
6,Normalized Net income,220.7 $,203.8 $,540.2 $,503.0
7,Normalized Net income attributable to sharehol...,200.6 $,165.2 $,476.2 $,396.9
8,Normalized Diluted EPS,3.59 $,2.96 $,8.54 $,7.0


## Table 2-4: Pag 7

Tabula is not detecting the first table of the page 7, so it will be extracted using a different method

### First table

In [396]:
import pdfplumber
import pandas as pd

# Open the PDF file
with pdfplumber.open(pdf_path) as pdf:

    page = pdf.pages[6] #BMO Report has table on Page 4

    text = page.extract_text()

print(text)

lines = text.split('\n')

data = []
headers = None

for line in lines:
    columns = line.split()

    if '(C$ in millions)' in line:
        headers = columns
    elif any(char.isdigit() for char in line.split()[0]):
        data.append(columns)

The following table reconciles Consolidated Normalized Income Before Income Taxes to Income
Before Income Taxes:
YTD YTD
(C$ in millions) Q3 2024 Q3 2023 Q3 2024 Q3 2023
Income before income taxes $ 299.3 $ 69.3 $ 716.9 $ 309.8
Add normalizing items:
DC fire expense (recovery) — (131.0) — 11.3
GST/HST-related charge — — — 33.3
Change in fair value of redeemable financial instrument — 328.0 328.0
Normalized Income before income taxes $ 299.3 $ 266.3 $ 716.9 $ 682.4
The following table reconciles Retail Normalized Income Before Income Taxes to Income Before
Income Taxes:
YTD YTD
(C$ in millions) Q3 2024 Q3 2023 Q3 2024 Q3 2023
Income before income taxes $ 299.3 $ 69.3 $ 716.9 $ 309.8
Less: Other operating segments 134.5 (169.7) 381.4 64.5
Retail Income before income taxes $ 164.8 $ 239.0 $ 335.5 $ 245.3
Add normalizing items:
DC fire expense (recovery) — (131.0) — 11.3
Retail Normalized Income before income taxes $ 164.8 $ 108.0 $ 335.5 $ 256.6
The following table reconciles Financial Se

In [397]:
print(lines[3:10])
text = lines[3:10]

['(C$ in millions) Q3 2024 Q3 2023 Q3 2024 Q3 2023', 'Income before income taxes $ 299.3 $ 69.3 $ 716.9 $ 309.8', 'Add normalizing items:', 'DC fire expense (recovery) — (131.0) — 11.3', 'GST/HST-related charge — — — 33.3', 'Change in fair value of redeemable financial instrument — 328.0 328.0', 'Normalized Income before income taxes $ 299.3 $ 266.3 $ 716.9 $ 682.4']


In [398]:
table_data = []
# Regex to extract the description and four dollar values or placeholders
pattern = r"^(.*?)(?:\s+\$?\s*(-?[\d.]+|—)\s+\$?\s*(-?[\d.]+|—)\s+\$?\s*(-?[\d.]+|—)\s+\$?\s*(-?[\d.]+|—))?$"

# Process each line in the text
for line in text[1:]:  # Skip the first line (header)
    match = re.match(pattern, line)
    if match:
        description = match.group(1).strip()  # Extract description
        values = [match.group(i).strip() if match.group(i) else "-" for i in range(2, 6)]  # Extract values
        table_data.append([description] + values)

# Create a DataFrame with appropriate column names
table2 = pd.DataFrame(table_data, columns=new_columns)
table2

,(C$ in millions),Q3_2024,Q3_2023,YTD_Q3_2024,YTD_Q3_2023
0,Income before income taxes,299.3,69.3,716.9,309.8
1,Add normalizing items:,-,-,-,-
2,DC fire expense (recovery) — (131.0) — 11.3,-,-,-,-
3,GST/HST-related charge,—,—,—,33.3
4,Change in fair value of redeemable financial i...,-,-,-,-
5,Normalized Income before income taxes,299.3,266.3,716.9,682.4


In [399]:
# Let's manually update the specific row 2
row_index = 2

# Manually split the data and assign values
table2.at[row_index, '(C$ in millions)'] = 'DC fire expense (recovery)'
table2.at[row_index, 'Q3_2024'] = '-'
table2.at[row_index, 'Q3_2023'] = '(131.0)'
table2.at[row_index, 'YTD_Q3_2024'] = '-'
table2.at[row_index, 'YTD_Q3_2023'] = '11.3'

# Let's manually update the specific row 2
row_index = 4

# Manually split the data and assign values
table2.at[row_index, '(C$ in millions)'] = 'DC fire expense (recovery)'
table2.at[row_index, 'Q3_2024'] = '-'
table2.at[row_index, 'Q3_2023'] = '328.0'
table2.at[row_index, 'YTD_Q3_2024'] = '-'
table2.at[row_index, 'YTD_Q3_2023'] = '328.0'

# Display the modified DataFrame
table2

,(C$ in millions),Q3_2024,Q3_2023,YTD_Q3_2024,YTD_Q3_2023
0,Income before income taxes,299.3,69.3,716.9,309.8
1,Add normalizing items:,-,-,-,-
2,DC fire expense (recovery),-,(131.0),-,11.3
3,GST/HST-related charge,—,—,—,33.3
4,DC fire expense (recovery),-,328.0,-,328.0
5,Normalized Income before income taxes,299.3,266.3,716.9,682.4


### Tables 3-4: Page 7

In [400]:
table3 = table[1]
table3

,Income before income taxes,$,299.3 $,69.3 $,716.9 $,309.8
0,Less: Other operating segments,NaN,134.5,(169.7),381.4,64.5
1,Retail Income before income taxes,$,164.8 $,239.0 $,335.5 $,245.3
2,Add normalizing items:,NaN,NaN,NaN,NaN,NaN
3,DC fire expense (recovery),NaN,—,(131.0),—,11.3
4,Retail Normalized Income before income taxes,$,164.8 $,108.0 $,335.5 $,256.6


In [401]:
table3.drop(columns="$", inplace=True)
table3

,Income before income taxes,299.3 $,69.3 $,716.9 $,309.8
0,Less: Other operating segments,134.5,(169.7),381.4,64.5
1,Retail Income before income taxes,164.8 $,239.0 $,335.5 $,245.3
2,Add normalizing items:,NaN,NaN,NaN,NaN
3,DC fire expense (recovery),—,(131.0),—,11.3
4,Retail Normalized Income before income taxes,164.8 $,108.0 $,335.5 $,256.6


In [402]:
new_columns = ['(C$ in millions)','Q3_2024', 'Q3_2023', 'YTD_Q3_2024', 'YTD_Q3_2023']
table3 = structure_table(table3, new_columns)
print("\nModified DataFrame:")
table3


Modified DataFrame:


,(C$ in millions),Q3_2024,Q3_2023,YTD_Q3_2024,YTD_Q3_2023
0,Income before income taxes,299.3 $,69.3 $,716.9 $,309.8
1,Less: Other operating segments,134.5,(169.7),381.4,64.5
2,Retail Income before income taxes,164.8 $,239.0 $,335.5 $,245.3
3,Add normalizing items:,NaN,NaN,NaN,NaN
4,DC fire expense (recovery),—,(131.0),—,11.3
5,Retail Normalized Income before income taxes,164.8 $,108.0 $,335.5 $,256.6


In [403]:
table4 = table[2]
table4

,Income before income taxes,$ 299.3,$ 69.3,Unnamed: 0,$ 716.9,$ 309.8
0,Less: Other operating segments,189.0,(56.4),NaN,422.4,10.0
1,Financial Services Income before income t...,$ 110.3,$ 125.7,NaN,$ 294.5,$ 299.8
2,Add normalizing items:,NaN,NaN,NaN,NaN,NaN
3,GST/HST-related charge,—,—,NaN,—,33.3
4,Financial Services Normalized Income befor...,$ 110.3,$ 125.7,NaN,$ 294.5,$ 333.1


In [404]:
table4.drop(columns="Unnamed: 0", inplace=True)
table4

,Income before income taxes,$ 299.3,$ 69.3,$ 716.9,$ 309.8
0,Less: Other operating segments,189.0,(56.4),422.4,10.0
1,Financial Services Income before income t...,$ 110.3,$ 125.7,$ 294.5,$ 299.8
2,Add normalizing items:,NaN,NaN,NaN,NaN
3,GST/HST-related charge,—,—,—,33.3
4,Financial Services Normalized Income befor...,$ 110.3,$ 125.7,$ 294.5,$ 333.1


In [405]:
new_columns = ['(C$ in millions)','Q3_2024', 'Q3_2023', 'YTD_Q3_2024', 'YTD_Q3_2023']
table4 = structure_table(table4, new_columns)
print("\nModified DataFrame:")
table4


Modified DataFrame:


,(C$ in millions),Q3_2024,Q3_2023,YTD_Q3_2024,YTD_Q3_2023
0,Income before income taxes,$ 299.3,$ 69.3,$ 716.9,$ 309.8
1,Less: Other operating segments,189.0,(56.4),422.4,10.0
2,Financial Services Income before income t...,$ 110.3,$ 125.7,$ 294.5,$ 299.8
3,Add normalizing items:,NaN,NaN,NaN,NaN
4,GST/HST-related charge,—,—,—,33.3
5,Financial Services Normalized Income befor...,$ 110.3,$ 125.7,$ 294.5,$ 333.1


## Table 5: Pag 8

In [406]:
table5 = table[3]
table5

,Income before income taxes,$,299.3 $,69.3,$.1,716.9 $,309.8
0,Less: Other operating segments,NaN,204.8,58.0,NaN,418.0,118.6
1,CT REIT income before income taxes,$,94.5 $,11.3,$,29 8.9$,191.2
2,Add:,NaN,NaN,NaN,NaN,NaN,NaN
3,CT REIT fair value loss (gain) adjustment,NaN,(17.7),66.7,NaN,(64.3),39.3
4,CT REIT deferred taxes,NaN,(0.6),(0.2),NaN,0.2,0.7
5,CT REIT lease principal payments on right-of-u...,NaN,(0.2),(0.2),NaN,(0.6),(0.7)
6,CT REIT fair value of equity awards,NaN,1.9,(0.9),NaN,0.7,(1.1)
7,CT REIT internal leasing expense,NaN,0.2,0.4,NaN,0.8,0.8
8,CT REIT funds from operations,$,78.1 $,77.1,$,235.7 $,230.2
9,Less:,NaN,NaN,NaN,NaN,NaN,NaN


In [407]:
table5.drop(columns=["$.1", "$"], inplace=True)
table5

,Income before income taxes,299.3 $,69.3,716.9 $,309.8
0,Less: Other operating segments,204.8,58.0,418.0,118.6
1,CT REIT income before income taxes,94.5 $,11.3,29 8.9$,191.2
2,Add:,NaN,NaN,NaN,NaN
3,CT REIT fair value loss (gain) adjustment,(17.7),66.7,(64.3),39.3
4,CT REIT deferred taxes,(0.6),(0.2),0.2,0.7
5,CT REIT lease principal payments on right-of-u...,(0.2),(0.2),(0.6),(0.7)
6,CT REIT fair value of equity awards,1.9,(0.9),0.7,(1.1)
7,CT REIT internal leasing expense,0.2,0.4,0.8,0.8
8,CT REIT funds from operations,78.1 $,77.1,235.7 $,230.2
9,Less:,NaN,NaN,NaN,NaN


In [408]:
new_columns = ['(C$ in millions)','Q3_2024', 'Q3_2023', 'YTD_Q3_2024', 'YTD_Q3_2023']
table5 = structure_table(table5, new_columns)
print("\nModified DataFrame:")
table5


Modified DataFrame:


,(C$ in millions),Q3_2024,Q3_2023,YTD_Q3_2024,YTD_Q3_2023
0,Income before income taxes,299.3 $,69.3,716.9 $,309.8
1,Less: Other operating segments,204.8,58.0,418.0,118.6
2,CT REIT income before income taxes,94.5 $,11.3,29 8.9$,191.2
3,Add:,NaN,NaN,NaN,NaN
4,CT REIT fair value loss (gain) adjustment,(17.7),66.7,(64.3),39.3
5,CT REIT deferred taxes,(0.6),(0.2),0.2,0.7
6,CT REIT lease principal payments on right-of-u...,(0.2),(0.2),(0.6),(0.7)
7,CT REIT fair value of equity awards,1.9,(0.9),0.7,(1.1)
8,CT REIT internal leasing expense,0.2,0.4,0.8,0.8
9,CT REIT funds from operations,78.1 $,77.1,235.7 $,230.2


## Table 6: Pag 9

In [409]:
table6 = tabula.read_pdf(pdf_path, pages=9, multiple_tables=False)

In [410]:
# Define the area manually for the part of the page that contains the table
table6 = tabula.read_pdf(pdf_path, pages=9, multiple_tables=False, stream=True)

table6 = table6[0]
table6


,(C$ in millions),Unnamed: 1,Q3 2024,Unnamed: 3,Q3 2023
0,Income before income taxes,$,979.8,$,"1,062.0"
1,Less: Other operating segments,NaN,482.6,NaN,174.3
2,Retail Income before income taxes,$,497.2,$,887.7
3,Add normalizing items:,NaN,NaN,NaN,NaN
4,Operational Efficiency program,NaN,—,NaN,19.5
5,Targeted headcount reduction-related charge,NaN,19.6,NaN,—
6,DC fire expense (recovery),NaN,—,NaN,11.3
7,Retail Normalized Income before income taxes,$,516.8,$,918.5
8,Less:,NaN,NaN,NaN,NaN
9,Retail intercompany adjustments1,NaN,216.7,NaN,213.7


In [411]:
table6.drop(columns=['Unnamed: 1','Unnamed: 3'], inplace=True)
table6

,(C$ in millions),Q3 2024,Q3 2023
0,Income before income taxes,979.8,"1,062.0"
1,Less: Other operating segments,482.6,174.3
2,Retail Income before income taxes,497.2,887.7
3,Add normalizing items:,NaN,NaN
4,Operational Efficiency program,—,19.5
5,Targeted headcount reduction-related charge,19.6,—
6,DC fire expense (recovery),—,11.3
7,Retail Normalized Income before income taxes,516.8,918.5
8,Less:,NaN,NaN
9,Retail intercompany adjustments1,216.7,213.7


## Table 7: Pag 10

In [412]:
table7 = table[5]
table7


,Total additions1,$,216.4,$.1,188.6,$.2,490.2 $,396.6
0,Add: Accrued additions,NaN,(21.3),NaN,(12.2),NaN,(32.6),39.2
1,Less: CT REIT acquisitions and developments,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,excluding vend-ins from CTC,NaN,68.0,NaN,21.3,NaN,82.0,42.7
3,Operating capital expenditures,$,127.1,$,155.1,$,375.6 $,393.1


In [413]:
table7.drop(columns=['$','$.1','$.2'], inplace=True)
table7

,Total additions1,216.4,188.6,490.2 $,396.6
0,Add: Accrued additions,(21.3),(12.2),(32.6),39.2
1,Less: CT REIT acquisitions and developments,NaN,NaN,NaN,NaN
2,excluding vend-ins from CTC,68.0,21.3,82.0,42.7
3,Operating capital expenditures,127.1,155.1,375.6 $,393.1


In [414]:
new_columns = ['(C$ in millions)','Q3_2024', 'Q3_2023', 'YTD_Q3_2024', 'YTD_Q3_2023']
table7 = structure_table(table7, new_columns)
print("\nModified DataFrame:")
table7


Modified DataFrame:


,(C$ in millions),Q3_2024,Q3_2023,YTD_Q3_2024,YTD_Q3_2023
0,Total additions1,216.4,188.6,490.2 $,396.6
1,Add: Accrued additions,(21.3),(12.2),(32.6),39.2
2,Less: CT REIT acquisitions and developments,NaN,NaN,NaN,NaN
3,excluding vend-ins from CTC,68.0,21.3,82.0,42.7
4,Operating capital expenditures,127.1,155.1,375.6 $,393.1


# Cleaning tables

In [415]:
def clean_tables(numeric_columns, table):
    table_df = table.copy()

    for col in numeric_columns:
        # Replace unwanted characters to prepare for numeric conversion
        table_df[col] = table_df[col].astype(str).str.replace('$', '', regex=False)
        table_df[col] = table_df[col].str.replace(',', '', regex=False)
        table_df[col] = table_df[col].str.replace('(', '-', regex=False)
        table_df[col] = table_df[col].str.replace(')', '', regex=False)

        # Convert all values to float, coercing errors into NaN
        table_df[col] = pd.to_numeric(table_df[col], errors='coerce')

        # Format numeric values
        table_df[col] = table_df[col].apply(
            lambda x: format_currency(x) if pd.notna(x) else '-'
        )

    return table_df


In [416]:
tables = [table1, table2, table3, table4, table5, table6, table7]

In [418]:
cleaned_tables = []

for table in tables:
    cleaned_tables.append(clean_tables(table.columns[1:], table))

In [423]:
cleaned_tables[4]

,(C$ in millions),Q3_2024,Q3_2023,YTD_Q3_2024,YTD_Q3_2023
0,Income before income taxes,$299,$69,$716,$309
1,Less: Other operating segments,$204,$58,$418,$118
2,CT REIT income before income taxes,$94,$11,-,$191
3,Add:,-,-,-,-
4,CT REIT fair value loss (gain) adjustment,-$17,$66,-$64,$39
5,CT REIT deferred taxes,-$0,-$0,$0,$0
6,CT REIT lease principal payments on right-of-u...,-$0,-$0,-$0,-$0
7,CT REIT fair value of equity awards,$1,-$0,$0,-$1
8,CT REIT internal leasing expense,$0,$0,$0,$0
9,CT REIT funds from operations,$78,$77,$235,$230
